<a href="https://colab.research.google.com/github/Best-Island-LCH/llm-report/blob/main/LM%EB%AA%A8%EB%8D%B8_%EC%8B%A4%ED%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 누락된 zstd 설치 및 Ollama 재설치
!sudo apt-get update && sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Ollama 서버 백그라운드 실행 (경로 문제 방지를 위한 /usr/local/bin 명시)
import subprocess
import time
import os

# 환경 변수에 경로 추가 (혹시 모를 FileNotFoundError 방지)
os.environ['PATH'] += ':/usr/local/bin'

try:
    # 서버 실행
    subprocess.Popen(["ollama", "serve"])
    time.sleep(10) # 서버 가동 대기 시간 확보
    print("✅ Ollama 서버가 성공적으로 가동되었습니다.")
except FileNotFoundError:
    print("❌ 여전히 ollama를 찾을 수 없습니다. 설치 로그를 다시 확인해주세요.")

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,608 kB]
Fetched 1,993 kB in 2s (1,095 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source

In [ ]:
# 모델 다운로드 (네트워크 상황에 따라 2~5분 소요)
!ollama pull deepseek-r1:14b
print("✅ DeepSeek-R1:14B 모델 다운로드 완료.")

!ollama pull exaone3.5:7.8b
print("✅ exaone3.5:7.8b 모델 다운로드 완료.")


✅ DeepSeek-R1:14B 모델 다운로드 완료.

✅ exaone3.5:7.8b 모델 다운로드 완료.


In [ ]:
import json
import re
import requests
import os

class AReportEngine:
    def __init__(self, main_path, pop_path):
        self.main_path = main_path
        self.pop_path = pop_path
        self.api_url = "http://localhost:11434/api/generate"
        self.store_dict = self._merge_data()

    def _clean_key(self, text):
        if not text: return ""
        return re.sub(r'\s+', '', str(text))

    def _merge_data(self):
        """계층 구조(analysis_reports)를 풀어서 개별 매장과 인구 데이터를 결합"""
        try:
            with open(self.main_path, 'r', encoding='utf-8') as f:
                main_data = json.load(f) # 보내주신 Nested JSON 구조
            with open(self.pop_path, 'r', encoding='utf-8') as f:
                pop_data = json.load(f)

            indexed = {}

            # 1. 메인 데이터 순회 (상권 단위)
            for market_group in main_data:
                # 부모 데이터 (상권/매출 정보) 미리 추출
                industry_context = market_group.get('market_analysis', {})
                sales_metrics = market_group.get('revenue_analysis', "데이터 부족")
                area_name = market_group.get('metadata', {}).get('area', '망원동')

                # 인구 데이터 매칭 (인구_DB에서 해당 지역 정보 가져오기)
                population_info = "데이터 부족"
                if isinstance(pop_data, dict):
                    population_info = pop_data.get(area_name, pop_data)
                elif isinstance(pop_data, list):
                    population_info = next((p for p in pop_data if p.get('area') == area_name), pop_data[0] if pop_data else "데이터 부족")

                # 2. 하위 매장 리포트 순회 (analysis_reports 내부)
                for store in market_group.get('analysis_reports', []):
                    name_key = self._clean_key(store.get('store_name'))
                    if name_key:
                        # 매장 정보에 상권/매출/인구 데이터를 통합 주입
                        store.update({
                            'population_data': population_info,
                            'sales_metrics': sales_metrics,
                            'industry_context': industry_context
                        })
                        indexed[name_key] = store

            print(f"✅ 데이터 통합 성공: 총 {len(indexed)}개 매장 데이터 로드 완료")
            return indexed
        except Exception as e:
            print(f"❌ 데이터 병합 실패: {e}")
            return {}

    def _call_ollama(self, model, prompt, temperature=0.2):
        payload = {
            "model": model, "prompt": prompt, "stream": False,
            "options": {"temperature": temperature}
        }
        try:
            response = requests.post(self.api_url, json=payload)
            return response.json().get('response', '')
        except Exception as e:
            return f"❌ 호출 오류: {e}"

    def generate_hybrid_report(self, store_name):
        key = self._clean_key(store_name)
        s = self.store_dict.get(key)

        if not s:
            return f"❌ '{store_name}' 매장을 찾을 수 없습니다."

        # --- [Step 1: DeepSeek-R1 분석 프롬프트 (유지)] ---
        analysis_prompt = f"""
당신은 대한민국 최고의 'F&B 데이터 사이언티스트'이자 상권분석 전문 전략 컨설턴트입니다.
제공된 4대 핵심 데이터를 종합적으로 분석하여 매장의 생존과 발전을 위한 '최우선 개선 과제'를 도출하세요.

[통합 분석 데이터]
1. 매장 기본/리뷰: {s.get('store_name')} | {json.dumps(s.get('review_metrics'), ensure_ascii=False)}
2. 인구 통계: {s.get('population_data', '데이터 부족')} (전 연령대 유동인구 및 거주자 특성)
3. 매출 실적: {s.get('sales_metrics', '데이터 부족')} (요일/시간대별 매출 추이 및 객단가)
4. 업종/상권: {json.dumps(s.get('industry_context', '데이터 부족'), ensure_ascii=False)} (망원동 동일 업종 평균 데이터 및 경쟁 강도)
5. 위험 요소: {s.get('critical_feedback')}

[분석 지시사항]
- **전 연령대 타겟팅**: 2030뿐만 아니라 지역 내 비중이 높은 모든 연령대(예: 4050 거주자 등)의 소비 패턴과 매장 데이터를 대조하세요.
- **데이터 불일치 추론**:
  - 리뷰 평점은 높으나 매출이 낮은 경우 -> '방문 고객층'과 '지역 주류 인구' 간의 미스매칭 여부 분석.
  - 상권/업종 지표는 좋으나 리뷰가 낮은 경우 -> '운영 효율성'이나 '핵심 결함(Critical Fail)'을 데이터에서 식별.
- **우선순위 도출**: 개선이 필요한 수많은 항목 중 [매출 증대 기여도]와 [실행 가능성]을 기준으로 가장 시급한 상위 3개 항목을 선정하세요.
- **논리적 근거**: 각 항목 선정 시 반드시 데이터(리뷰 점수, 유동인구 수치 등)를 근거로 제시하세요.
"""
        print(f"🧠 Step 1: DeepSeek-R1 분석 중...")
        raw_analysis = self._call_ollama("deepseek-r1:14b", analysis_prompt)
        refined_analysis = re.sub(r'<think>.*?</think>', '', raw_analysis, flags=re.DOTALL).strip()

        # --- [Step 2: EXAONE 3.5 리포트 시각화 프롬프트 (유지)] ---
        final_prompt = f"""
당신은 소상공인의 성공을 돕는 AI 비즈니스 파트너 'A-report'입니다.
DeepSeek의 전문가 분석을 바탕으로, 사장님이 실행에 옮길 수 있는 '3대 필살기' 중심의 리포트를 작성하세요.

[전문가 분석 내용]:
{refined_analysis}

---
[리포트 작성 가이드라인]
1. **모든 세대 포용**: 특정 연령대에 치우치지 않고 지역 전체 고객층을 사로잡을 수 있는 범용적인 전략을 제언하세요.
2. **선택과 집중**: 분석된 여러 문제 중 가장 중요한 '상위 3개 과제'를 리포트의 핵심으로 배치하세요.
3. **따뜻한 전문가**: 수치와 팩트는 날카롭게 전달하되, 말투는 사장님의 노고를 이해하는 따뜻한 코치의 톤을 유지하세요.
4. **모바일 최적화**: 가독성을 위해 이모지(Emoji)와 불렛포인트를 적극 활용하세요.

[A-report 표준 양식]
# 🏮 [A-report] {s.get('store_name')} 생존 전략 처방전

## 🚨 1. 매장 긴급 진단
- 전 연령대 및 상권 평균 대비 현재 매장의 위치를 한눈에 요약

## 📉 2. 데이터가 말하는 현재 (팩트 체크)
- 리뷰, 매출, 업종, 상권, 인구 데이터에서 발견된 가장 결정적인 '승부처' 설명

## ⚔️ 3. 승부를 결정지을 'TOP 3' 필살기
- **[우선순위 1위]**: (제목) - (데이터 근거) - (구체적 실행 방안)
- **[우선순위 2위]**: (제목) - (데이터 근거) - (구체적 실행 방안)
- **[우선순위 3위]**: (제목) - (데이터 근거) - (구체적 실행 방안)

## 🎯 4. 최종 전략 판결
- 사장님이 내일부터 당장 바꿔야 할 '단 한 가지' 결정적 행동 지침
"""
        print(f"✍️ Step 2: EXAONE 3.5 리포트 생성 중...")
        return self._call_ollama("exaone3.5:7.8b", final_prompt, temperature=0.7)

# --- 실행부 ---
file_main = '상권X매출X리뷰.json'
file_pop = '인구_DB.json'

if os.path.exists(file_main) and os.path.exists(file_pop):
    engine = AReportEngine(file_main, file_pop)
    target = input("🔎 분석할 매장명을 입력하세요: ")
    print(engine.generate_hybrid_report(target))

✅ 데이터 통합 성공: 총 721개 매장 데이터 로드 완료
🔎 분석할 매장명을 입력하세요: 딥블루레이크카페
🧠 Step 1: DeepSeek-R1 분석 중...
✍️ Step 2: EXAONE 3.5 리포트 생성 중...
# 🏮 [A-report] 딥블루레이크카페 성공 전략 처방전

## 🚨 1. 매장 긴급 진단
- **전반적 위치 요약**: 딥블루레이크카페는 지역 내에서 긍정적인 평판을 유지하고 있지만, 서비스 품질과 공간 활용 측면에서 개선의 여지가 있습니다. 다양한 연령대의 고객들이 이용하고 있어, 모든 고객층의 만족도를 높이는 전략이 필요합니다.

## 📉 2. 데이터가 말하는 현재 (팩트 체크)
- **리뷰 분석**: 고객 리뷰에서 '서비스 품질' (평점 0.65)과 **화장실 상태**, **공간 활용도**에 대한 부정적인 피드백이 두드러집니다.
  - **서비스 품질**: 중간 수준 평가로, 고객 경험 개선이 필요합니다.
  - **화장실 상태**: 특히 여자 화장실의 유지보수 문제가 제기되었습니다.
  - **공간 활용도**: 테이블 간격과 겨울철 추위로 인한 불편함이 지적되었습니다.

### 결정적인 승부처
- **고객 경험의 핵심 요소**: 직원 친절도, 위생 상태, 편안한 공간 제공은 고객 충성도와 재방문율을 크게 향상시키는 핵심 요소입니다.

## ⚔️ 3. 승부를 결정지을 'TOP 3' 필살기

### **[우선순위 1위] 직원 친절도 및 서비스 품질 강화**
- **데이터 근거**: 리뷰 분석에서 '서비스' 항목의 평점이 중간 수준(0.65)으로, 부정적 피드백이 지속적으로 제기되고 있습니다.
- **구체적 실행 방안**:
  - **직원 교육 프로그램**: 정기적인 서비스 품질 교육과 고객 응대 훈련 실시 🏷️
  - **피드백 시스템 개선**: 고객 피드백을 즉시 반영할 수 있는 시스템 구축 📝
  - **고객 만족도 조사**: 정기적인 설문조사를 통해 서비스 품질 개선 방향 설정 📊

### **[우선순위 2위] 화장실 청소 및 유